In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import seaborn as sns
sns.set_style('white')
import re
from shapely.geometry import Point
import geopandas as gpd
import folium

In [2]:
df = pd.read_excel('/Users/khani26/Dropbox/tube.xlsx')
ignore = ['Auto top-up', 'Topped up', 'Topped-up', 'Bus journey', 'Entered']
df = df[~df.journey.str.contains('|'.join(ignore))].reset_index(drop=True)

In [3]:
df.head(2)

,date,start,end,journey,charge,credit,balance,note
0,2017-11-15,16:44:00,16:56:00,High Street Kensington to Edgware Road (Circle...,2.4,NaN,NaN,Contactless
1,2017-11-15,09:24:00,09:35:00,"Edgware Road (Circle, District lines) to High ...",2.4,NaN,NaN,Contactless


In [4]:
df.end.fillna('00:00:00', inplace=True)

df['start_time'] = (df.date.astype(str) + ' ' + df.start.astype(str)).astype('datetime64[ns]')
df['end_time'] = (df.date.astype(str) + ' ' + df.end.astype(str)).astype('datetime64[ns]')
df['duration'] = (df.end_time - df.start_time).apply(lambda x: pd.Timedelta(0) if x.days != 0 else x)
df['minutes'] = df.duration.apply(lambda x: x.components.minutes + 60*x.components.hours)

df['journey'] = df.journey.apply(lambda x: re.sub("[\(\[].*?[\)\]]", "", x))
df['start_station'] = df.journey.apply(lambda x: x.split(' to ')[0].strip())
df['end_station'] = df.journey.apply(lambda x: x.split(' to ')[-1].strip())

In [6]:
df.date.dt.weekday_name.value_counts()

Friday       158
Wednesday    127
Tuesday      121
Monday       107
Thursday     101
Sunday        84
Saturday      70
Name: date, dtype: int64

In [7]:
stations = pd.read_csv('stations.csv')

In [8]:
stations['location'] = [Point(i) for i in zip(stations.lat, stations.lon)]
del stations['lat']
del stations['lon']

stations = gpd.GeoDataFrame(stations, geometry='location', crs={'init' :'epsg:4326'})

In [9]:
df = df.merge(stations, how='left', left_on='start_station', right_on='station')
df = df.merge(stations, how='left', left_on='end_station', right_on='station', suffixes=('_start', '_end'))
del df['station_start']
del df['station_end']

In [10]:
london = gpd.read_file('london/London-wards-2014_ESRI/London_Ward_CityMerged.shp')
london = london.to_crs(stations.crs)

In [18]:
m = folium.Map(tiles='cartodbpositron', location=(51.5, 0))

for i in df.drop_duplicates(subset=['end_station']).itertuples():
    folium.Marker(location = i.location_end.coords[0],
                  popup = i.end_station,
                  icon = folium.Icon(color='red')).add_to(m)

m

In [17]:
m.save('index.html')

In [23]:
df.head()

,date,start,end,journey,charge,credit,balance,note,start_time,end_time,duration,minutes,start_station,end_station,zone_start,location_start,zone_end,location_end
0,2017-11-15,16:44:00,16:56:00,High Street Kensington to Edgware Road,2.4,NaN,NaN,Contactless,2017-11-15 16:44:00,2017-11-15 16:56:00,00:12:00,12,High Street Kensington,Edgware Road,1,POINT (51.50072945 -0.191566058),1,POINT (51.51999804 -0.167668254)
1,2017-11-15,09:24:00,09:35:00,Edgware Road to High Street Kensington,2.4,NaN,NaN,Contactless,2017-11-15 09:24:00,2017-11-15 09:35:00,00:11:00,11,Edgware Road,High Street Kensington,1,POINT (51.51999804 -0.167668254),1,POINT (51.50072945 -0.191566058)
2,2017-10-25,20:59:00,21:15:00,Victoria to Marylebone,2.4,NaN,NaN,Contactless,2017-10-25 20:59:00,2017-10-25 21:15:00,00:16:00,16,Victoria,Marylebone,1,POINT (51.49642395 -0.143920706),1,POINT (51.52239667 -0.163492564)
3,2017-10-25,17:55:00,18:17:00,Paddington to Victoria,2.4,NaN,NaN,Contactless,2017-10-25 17:55:00,2017-10-25 18:17:00,00:22:00,22,Paddington,Victoria,1,POINT (51.51539379 -0.175736752),1,POINT (51.49642395 -0.143920706)
4,2017-10-12,20:57:00,21:08:00,Kings Cross St Pancras to Baker Street,2.4,NaN,NaN,Contactless,2017-10-12 20:57:00,2017-10-12 21:08:00,00:11:00,11,Kings Cross St Pancras,Baker Street,1,POINT (51.53031246 -0.12387213),1,POINT (51.52312962 -0.156904194)


In [19]:
df.groupby(['start_station', 'end_station']).agg({'minutes': ['mean', 'count'],
                                                  'charge': 'mean'}).reset_index()

start_station          end_station    charge    minutes      
                                               mean       mean count
0       Aldgate East        Goldhawk Road  1.500000  42.000000     1
1       Aldgate East      West Kensington  2.000000  31.000000     2
2              Angel          Earls Court  1.250000  32.000000     1
3              Angel           Marylebone  2.300000  23.000000     1
4       Baker Street           Farringdon  0.600000  12.000000     1
5       Baker Street        Goldhawk Road  0.750000  21.000000     4
6       Barons Court              Holborn  1.550000  24.000000     2
7      Bethnal Green           Marylebone  0.600000  24.000000     1
8        Blackfriars          Earls Court  1.600000  20.000000     1
9       Boston Manor          Earls Court  0.400000  28.500000     6
10      Canary Wharf        London Bridge  0.600000   9.000000     1
11     Chancery Lane        Oxford Circus  0.600000  11.000000     1
12     Chancery Lane      Shepherd's Bush  1.750000  22.000000     2
13     Covent Garden          Manor House  0.600000  20.000000     1
14     Covent Garden     South Kensington  0.600000  16.000000     1
15     Covent Garden      West Kensington  1.900000  22.000000     1
16     Ealing Common          Earls Court  0.400000  18.000000     1
17       Earls Court         Baker Street  1.600000  21.000000     1
18       Earls Court             Barbican  0.600000  33.000000     1
19       Earls Court        Bethnal Green  0.600000  36.000000     1
20       Earls Court          Blackfriars  0.600000  21.000000     1
21       Earls Court         Boston Manor  0.422222  26.222222     9
22       Earls Court        Covent Garden  1.600000  21.000000     1
23       Earls Court             East Ham  0.700000  64.000000     1
24       Earls Court         Edgware Road  1.261364  17.681818    22
25       Earls Court           Embankment  0.975000  18.000000     8
26       Earls Court          Hammersmith  0.912000   9.000000    25
27       Earls Court        London Bridge  2.400000  23.000000     1
28       Earls Court           Marylebone  1.268750  28.500000     8
29       Earls Court           Old Street  0.600000  33.000000     1
..               ...                  ...       ...        ...   ...
125           Temple          Hammersmith  1.583333  27.875000    24
126           Temple             Monument  0.600000  17.000000     1
127           Temple             Moorgate  0.600000  18.000000     1
128       Tower Hill     South Kensington  0.600000  30.000000     1
129         Vauxhall          Earls Court  1.620000  19.400000     5
130         Vauxhall           Marylebone  1.450000  17.000000     2
131         Victoria           Marylebone  2.400000  16.000000     1
132  Wandsworth Road   Kensington Olympia  0.400000  46.000000     1
133         Waterloo         Boston Manor  0.700000  39.000000     1
134         Waterloo          Earls Court  0.750000  22.950000    20
135         Waterloo     Leicester Square  0.600000   9.000000     1
136         Waterloo        London Bridge  0.600000   8.000000     1
137         Waterloo  Mornington Crescent  0.750000  17.000000     1
138         Waterloo      North Greenwich  0.750000  16.000000     1
139  West Kensington         Aldgate East  1.500000  34.000000     1
140  West Kensington                Angel  2.300000  39.000000     1
141  West Kensington        Covent Garden  0.600000  24.000000     1
142  West Kensington      Gloucester Road  0.600000   8.166667     6
143  West Kensington        Goldhawk Road  1.666667  14.833333     6
144  West Kensington          Kings Cross  2.900000  33.000000     1
145  West Kensington        Mansion House  2.000000  29.000000     2
146  West Kensington           Marylebone  1.393750  30.312500    16
147  West Kensington           Paddington  2.300000  22.000000     1
148  West Kensington    Prince Regent DLR  0.850000  44.000000     1
149  West Kensington     South Kensington  0.600000  10.958333    24
150  